In [8]:
import pymssql
import pandas as pd
import configparser

In [9]:
config = configparser.ConfigParser()
config.read('db_config.ini')

db_host = config['database']['host']
db_user = config['database']['user']
db_password = config['database']['password']
db_name = config['database']['dbname']

sql_conn = pymssql.connect(db_host, db_user, db_password, db_name)

In [10]:
def get_data(query, columns = None):
    """Query in SQL STRING
    Columns = List of strings representing the columns for the df"""
    mycursor = sql_conn.cursor()
    mycursor.execute(query)
    result = mycursor.fetchall()
    data_list = list(result)
    if columns == None:
        df = pd.DataFrame(data_list)
        return df
    else:
        df = pd.DataFrame(data_list, columns= columns)
        return df

In [11]:
#Find all events where opioid dependence was diagnosed

query_diagnosis_columns = """SELECT *
FROM syscolumns
WHERE id=OBJECT_ID('CDW_NEW.deid_uf.DiagnosisEventFact') """
diagnosis_columns = get_data(query_diagnosis_columns)[0].to_list()
query_opioid = """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
WHERE DiagnosisName LIKE '%opioid%'
AND DiagnosisName LIKE '%dependence%'
AND DiagnosisName NOT LIKE '%excluding opioid%'
"""
# query_ms= """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
# WHERE DiagnosisName LIKE '%multiple sclerosis%' """
opioid_diagnosis = get_data(query_opioid, diagnosis_columns)
opioid_diagnosis

,DeidLds,PatientDurableKey,DiagnosisEventKey,DiagnosisKey,DiagnosisName,PatientKey,AgeKey,AgeKeyValue,EncounterKey,DepartmentKey,...,DocumentedByProviderType,DocumentedByProviderPrimarySpecialty,DocumentedByProviderDurableKey,Type,Status,PresentOnAdmission,HospitalDiagnosis,EmergencyDepartmentDiagnosis,Chronic,Count
0,deid_uf,DE43CDC63B2ADA,D2FCA66023425A,635210,"Opioid dependence, in remission (CMS code)",DB7F7759C8E4A1,15692004.0,NaN,DBAA51064CE1CB,24297,...,Medical Student,,981162,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
1,deid_uf,DFF15C4FFDB2AF,D7A8CDC8D582C4,1194004,Polysubstance dependence including opioid drug...,DB99A579E98633,-1.0,NaN,DB0E0F931DF4B3,27,...,Medical Student,,765792,Medical History,Active,*Not Applicable,NaN,NaN,NaN,1
2,deid_uf,D5136A28B79CB5,DCC3B8CA3E7196,336157,Polysubstance dependence including opioid type...,DFC730659F9E1F,15192004.0,NaN,-1,-1,...,Medical Student,,751630,Problem List,Active,*Unspecified,NaN,NaN,0.0,1
3,deid_uf,D38F5FBD6864E0,D5846CD1460D59,630317,Opioid dependence with withdrawal (CMS code),DDCD053FB18E3D,10228004.0,NaN,D2B89CF1625D10,43,...,Physician,Emergency Medicine,197964,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
4,deid_uf,D950B4EB26351E,D0C0C934C45810,630317,Opioid dependence with withdrawal (CMS code),D6926A65174695,17441004.0,155.0,D9392BD087C0B6,43,...,Physician,Emergency Medicine,197964,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47358,deid_uf,DC236CFA5EBC3A,D6D38F7CF3655D,90185,"Opioid dependence, uncomplicated (CMS code)",D22BFF068C2032,NaN,NaN,D9F95C9F7FA32F,43,...,Physician,Emergency Medicine,195174,Billing Diagnosis,Active,*Not Applicable,NaN,NaN,NaN,1
47359,deid_uf,D5BF5585BE17F9,D37FDF8F2C46A5,247835,Opioid dependence (CMS code),D6B691C74AA9B3,-1.0,NaN,D5D19AED1D182E,695,...,Physician,Critical Care Medicine,211040,Medical History,Active,*Not Applicable,NaN,NaN,NaN,1
47360,deid_uf,DD26462A0B3AB8,D0A1F07D8DBA77,85840,Opioid type dependence (CMS code),DB93D87D259ADB,-1.0,NaN,DFB6E9C928BF94,73,...,Physician,Critical Care Medicine,200900,Medical History,Active,*Not Applicable,NaN,NaN,NaN,1
47361,deid_uf,D22DD009429681,DE201BD90ACD96,247835,Opioid dependence (CMS code),DBA5A47E5D14A2,12281004.0,671.0,D8AB365CC2C0E9,3427,...,Physician,Internal Medicine,233017,Hospital Problem,Active,Unknown,1.0,NaN,0.0,1


In [12]:
type(opioid_diagnosis["PatientDurableKey"])

pandas.core.series.Series

In [13]:
#Find all events where opioids were prescribed to patients— limited to oxycontin and oxycodone

# dependent_record_keys = opioid_diagnosis["PatientDurableKey"].tolist()
# dependent_record_keys_str = ', '.join([str(key) for key in dependent_record_keys])

query_med_columns = """SELECT *
FROM syscolumns
WHERE id=OBJECT_ID('CDW_NEW.deid_uf.MedicationOrderFact') """
med_columns = get_data(query_med_columns)[0].to_list()
query_opioid_presc = f"""
SELECT * 
FROM CDW_NEW.deid_uf.MedicationOrderFact
WHERE (MedicationName LIKE '%oxycontin%'
OR MedicationName LIKE '%oxycodone%')
"""
opioid_prescribed = get_data(query_opioid_presc, med_columns)
opioid_prescribed

,DeidLds,PatientDurableKey,MedicationOrderKey,PatientKey,EncounterKey,DepartmentKey,BedName,DepartmentName,DepartmentSpecialty,PharmacyKey,...,ReorderedFromPrescription,Verbal,RequiresCosign,Cosigned,VerbalSigned,Protocol,HighPriority,Prn,Count,SupervisedByProviderDurableKey
0,deid_uf,D38B708B0429FC,D3871047CA10E2,D8551E8144E34D,DFDBEAD5B4D64F,17399,*Not Applicable,BREAST MED ONC MB 3,Breast Care - Cancer Center,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
1,deid_uf,D98EEFDB5C4552,D1A81DBEAEF3EB,D7DE6F21A7A410,D250586A258449,2378,*Not Applicable,C4 PICU MB,Inpatient Nursing,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
2,deid_uf,DBD0B675678482,D898E49D7328AE,DFB9FB5286ED3B,DD07DD0D3ABBB1,13,*Not Applicable,8 NICU,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
3,deid_uf,DF78BDB5AC9A47,DB83EB2415394C,D8013041517D01,D30AD68F3FB41E,2380,*Not Applicable,C6 HEME/ONC MB,Inpatient Nursing,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
4,deid_uf,D1ED51861364A9,D85F44ED3B4C72,D89CBEA312D93C,D64E18BE1B34BB,26,*Not Applicable,14L MEDICINE,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313980,deid_uf,D9D2D6D4BE60EC,DB36B2E188BA96,D91054C5F9D93A,D0201229FDF97A,2385,*Not Applicable,A5 ONC MB,Oncology,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
1313981,deid_uf,D8A4BA0B50C693,DFA93DC1A82E12,D2209C3308EE39,DDBF7AC03BBA5F,2768,*Not Applicable,PPU,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
1313982,deid_uf,D8A4BA0B50C693,D464D423DB6918,D2209C3308EE39,DDBF7AC03BBA5F,2768,*Not Applicable,PPU,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1
1313983,deid_uf,DD555BF574B322,D07C1942078202,DF60C861FBB338,DEBF33009EC09B,2768,*Not Applicable,PPU,*Unspecified,-2,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,1,-1


In [7]:
query_opioid_presc

"\nSELECT * \nFROM CDW_NEW.deid_uf.MedicationOrderFact\nWHERE (MedicationName LIKE '%oxycontin%'\nOR MedicationName LIKE '%oxycodone%')\n"